In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm
import builtins

In [ ]:
# Ensure output directory exists, if not, create it, if fail, throw error
try:
    os.makedirs(output_path, exist_ok=True)
except OSError as e:
    raise Exception(f"Error creating models directory: {e.strerror}")

In [ ]:
# Check if builtins.input is set
if not hasattr(builtins, 'input'):
    raise AttributeError("No trading input found in `builtins.input`. Set it dynamically before running.")

In [ ]:
# Extract trading input from builtins
data = builtins.data
required_columns = ['prices', 'Action', 'Trade_Percentage']
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Column '{col}' is missing in the trading DataFrame.")

In [ ]:
# Parameters
initial_capital = 10000.0  # Starting USD balance
initial_btc_balance = 0.0  # Starting BTC balance
maker_fee = 0.0025  # Buy fee
taker_fee = 0.004  # Sell fee

In [ ]:
# Simulate trades
usd_balance = initial_capital
btc_balance = initial_btc_balance

In [ ]:
# Create or overwrite the trading log file
csv_file_path = f"{output_path}trading_log.csv"

In [ ]:
# Add any additional calculated columns to the DataFrame
data['Total_Capital'] = data['USD_Balance'] + data['BTC_Balance'] * data['prices']

In [ ]:
# Replace NaNs in Action column with "None"
data['Action'] = data['Action'].fillna('None')

In [ ]:
print("Trading Log:")
display(data)

In [ ]:
# Write the DataFrame directly to a CSV file
data.to_csv(csv_file_path, index=False)

In [ ]:
# Read the CSV file
trading_data = pd.read_csv(csv_file_path)

In [ ]:
# Iterate over actions and simulate trades
def process_trades(trading_data, usd_balance, btc_balance, maker_fee, taker_fee, show_progress=True):
    """
    Process trades based on the trading input.

    Args:
        trading_data (pd.DataFrame): DataFrame containing trading actions, percentages, and prices.
        usd_balance (float): Initial USD balance.
        btc_balance (float): Initial BTC balance.
        maker_fee (float): Fee for making a trade (buying).
        taker_fee (float): Fee for taking a trade (selling).
        show_progress (bool): Whether to show progress bars using tqdm.

    Returns:
        tuple: Updated balances, histories, and trade counts:
            - usd_balance (float): Final USD balance.
            - btc_balance (float): Final BTC balance.
            - total_capital_history (list): List of total portfolio values over time.
            - usd_balance_history (list): List of USD balances over time.
            - btc_balance_history (list): List of BTC balances over time.
            - buy_count (int): Total number of buy trades executed.
            - sell_count (int): Total number of sell trades executed.
    """
    total_capital_history = []
    usd_balance_history = []
    btc_balance_history = []
    buy_count = 0
    sell_count = 0

    # Wrap tqdm around iterrows only if show_progress is True
    row_iterator = tqdm(trading_data.iterrows(), desc="Processing Rows", unit="rows", total=len(trading_data)) if show_progress else trading_data.iterrows()

    for index, row in row_iterator:
        action = row['Action']
        percentage = row['Trade_Percentage']  # Percentage to buy or sell, as determined by the model
        price = row['prices']  # Use the price directly from trading_data

        if action == 'Buy' and usd_balance > 0:
            # Calculate the amount to invest based on the percentage
            amount_to_invest = usd_balance * percentage
            if amount_to_invest > 1e-6:  # Avoid negligible trades
                fee = amount_to_invest * maker_fee
                btc_bought = (amount_to_invest - fee) / price  # Deduct fee before buying
                usd_balance -= amount_to_invest
                btc_balance += btc_bought
                buy_count += 1

        elif action == 'Sell' and btc_balance > 0:
            # Calculate the amount of BTC to sell based on the percentage
            btc_to_sell = btc_balance * percentage
            if btc_to_sell > 1e-6:  # Avoid negligible trades
                usd_gained = btc_to_sell * price
                fee = usd_gained * taker_fee
                usd_gained -= fee  # Deduct fee after selling
                btc_balance -= btc_to_sell
                usd_balance += usd_gained
                sell_count += 1

        # Update histories
        total_capital_history.append(usd_balance + btc_balance * price)
        usd_balance_history.append(usd_balance)
        btc_balance_history.append(btc_balance)

    return usd_balance, btc_balance, total_capital_history, usd_balance_history, btc_balance_history, buy_count, sell_count

In [ ]:
# Process trades
usd_balance, btc_balance, total_capital_history, usd_balance_history, btc_balance_history, buy_count, sell_count = process_trades(
    trading_data=trading_data,
    usd_balance=initial_capital,
    btc_balance=0.0,
    maker_fee=0.0025,
    taker_fee=0.0040
)

In [ ]:
# Calculate final portfolio values
final_btc_price = trading_data['prices'].iloc[-1]
remaining_btc_value = btc_balance * final_btc_price
total_portfolio_value = usd_balance + remaining_btc_value
profit_loss = ((total_portfolio_value - initial_capital) / initial_capital) * 100

In [ ]:
# Display results
# results = {
#     "Final USD Balance": f"${usd_balance:.2f}",
#     "Remaining BTC": f"{btc_balance:.6f} BTC",
#     "Remaining BTC Value (USD)": f"${remaining_btc_value:.2f}",
#     "Final Portfolio Value (USD)": f"${total_portfolio_value:.2f}",
#     "Profit/Loss": f"{profit_loss:.2f}%",
#     "Total Trades Executed": buy_count + sell_count,
#     "Buy Trades": buy_count,
#     "Sell Trades": sell_count
# }
# print("Trading Simulation Results:")
# display(pd.DataFrame([results]))

# Print the final portfolio status
print("Final Portfolio Status:")
print(f"  USD Balance: ${usd_balance:.2f}")
print(f"  BTC Balance: {btc_balance:.6f} BTC")
print(f"  BTC Value (in USD at last price): ${remaining_btc_value:.2f}")
print(f"  Total Portfolio Value (USD): ${total_portfolio_value:.2f}")
print(f"  Profit/Loss: {profit_loss:.2f}%")
print(f"  Total Trades Executed: {buy_count + sell_count}")
print(f"    Buy Trades: {buy_count}")
print(f"    Sell Trades: {sell_count}")

In [ ]:
# Ensure required columns exist in trading_data
required_columns = ['prices', 'Action']
for col in required_columns:
    if col not in trading_data.columns:
        raise ValueError(f"Column '{col}' is missing in trading_data.")

# Plot the prices and actions using trading_data
plt.figure(figsize=(12, 6))

# Plot prices from trading_data
plt.plot(trading_data['prices'], label='Prices', color='blue', alpha=0.7, linewidth=1.5)

# Highlight buy and sell actions
buy_indices = trading_data[trading_data['Action'] == 'Buy'].index
sell_indices = trading_data[trading_data['Action'] == 'Sell'].index

# Scatter buy and sell actions on the price plot
plt.scatter(buy_indices, trading_data.loc[buy_indices, 'prices'], color='green', label='Buy', marker='^', s=60, alpha=0.8)
plt.scatter(sell_indices, trading_data.loc[sell_indices, 'prices'], color='red', label='Sell', marker='v', s=60, alpha=0.8)

# Final plot adjustments
plt.title('Prices with Buy/Sell Actions', fontsize=14)
plt.xlabel('Time Steps', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate cumulative moving average for a smooth curve
cumulative_average_total = pd.Series(total_capital_history).expanding(min_periods=1).mean()
cumulative_average_usd = pd.Series(usd_balance_history).expanding(min_periods=1).mean()
cumulative_average_btc = pd.Series(btc_balance_history).expanding(min_periods=1).mean()

In [ ]:
# Plot portfolio value progression
time_indices = range(len(total_capital_history))
plt.figure(figsize=(12, 6))
plt.plot(time_indices, cumulative_average_total, label="Total Portfolio Value", color='blue')
plt.title("Portfolio Value Over Time", fontsize=14)
plt.xlabel("Time Steps", fontsize=12)
plt.ylabel("Value (USD)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot USD balance progression
time_indices = range(len(usd_balance_history))
plt.figure(figsize=(12, 6))
plt.plot(time_indices, cumulative_average_usd, label="USD Balance", color='green', linewidth=2)
plt.axhline(y=initial_capital, color='gray', linestyle='--', label="Initial Balance (10k USD)")
plt.title("USD Balance Progression Over Time", fontsize=14)
plt.xlabel("Time Steps", fontsize=12)
plt.ylabel("USD Balance (in USD)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot BTC balance progression
time_indices = range(len(btc_balance_history))
plt.figure(figsize=(12, 6))
plt.plot(time_indices, cumulative_average_btc, label="BTC Balance", color='orange', linewidth=2)
plt.title("BTC Balance Progression Over Time", fontsize=14)
plt.xlabel("Time Steps", fontsize=12)
plt.ylabel("BTC Balance (in BTC)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Free-up memory by clean and close plt
plt.close('all')